# REST API Data Source Demo

Load paginated JSON endpoints by registering the spark-fuse REST data source, building an
options payload with `build_rest_api_config`, and reading via `spark.read.format`.

In [1]:
from spark_fuse.spark import create_session
from spark_fuse.io import (
    REST_API_CONFIG_OPTION,
    REST_API_FORMAT,
    build_rest_api_config,
    register_rest_data_source,
)

spark = create_session(app_name="rest-api-demo")
register_rest_data_source(spark)

:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/kevin/.ivy2.5.2/cache
The jars for the packages stored in: /Users/kevin/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-958327d7-f4ee-4a7e-a193-527a66cb2d15;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 84ms :: artifacts dl 16ms
	:: modules in use:
	io.delta#delta-spark_2.13;4.0.0 from central in [default]
	io.delta#delta-storage;4.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.13.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules      

In [2]:
import json

config = build_rest_api_config(
    spark,
    "https://pokeapi.co/api/v2/pokemon",
    source_config={
        "request_type": "GET",
        "records_field": "results",
        "pagination": {"mode": "response", "field": "next", "max_pages": 2},
        "params": {"limit": 20},
        "headers": {"User-Agent": "spark-fuse-demo/1.0 (contact@example.com)"},
    },
)

pokemon = (
    spark.read.format(REST_API_FORMAT)
    .option(REST_API_CONFIG_OPTION, json.dumps(config))
    .load()
)
pokemon.select("name").show(5)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py:161: DeprecationWarning: This process (pid=13478) is multi-threaded, use of fork() may lead to deadlocks in the child.


+----------+
|      name|
+----------+
| bulbasaur|
|   ivysaur|
|  venusaur|
|charmander|
|charmeleon|
+----------+
only showing top 5 rows
